In [1]:
import os
import numpy as np
import tensorflow as tf
from scipy.io import loadmat,savemat

feature_dimension = 90
frames = 100
fc_neurons = 256
tf.set_random_seed(3456)

In [2]:
x = tf.placeholder(shape = [None, feature_dimension, frames], dtype = tf.float32) #inputs
print(x.shape)
y = tf.placeholder(shape = [None, 2], dtype = tf.float32) #labels

(?, 90, 100)


In [3]:
#conv layer 1
weights_conv1 = tf.get_variable("conv1_weights",
                                shape = [3,3,1,256],  
                                dtype = tf.float32,
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv1 = tf.get_variable("conv1_bias",
                             shape = [256],
                              dtype = tf.float32,
                              initializer = tf.contrib.layers.xavier_initializer())

#reshaping of input(batch_size, feature_width, feature_height,input_channels)
input_x = tf.reshape(x,[-1,feature_dimension, frames,1])
print(input_x)

conv1 = tf.nn.conv2d(input_x, weights_conv1, strides = [1,2,2,1], padding = 'VALID')
conv1_out = tf.nn.relu(conv1)
print(conv1_out)


Tensor("Reshape:0", shape=(?, 90, 100, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 44, 49, 256), dtype=float32)


In [4]:
# conv layer 2
weights_conv2 = tf.get_variable("conv2_weights",
                               shape = [3,3,256,128],
                               dtype = tf.float32,
                                initializer = tf.contrib.layers.xavier_initializer())
bias_conv2 = tf.get_variable("conv2_bias",
                             shape = [128],
                            dtype = tf.float32,
                            initializer = tf.contrib.layers.xavier_initializer())

conv2 = tf.nn.conv2d(conv1_out,weights_conv2, strides = [1,2,2,1], padding = 'VALID')
conv2_out = tf.nn.relu(conv2)
print(conv2_out)

Tensor("Relu_1:0", shape=(?, 21, 24, 128), dtype=float32)


In [5]:
# conv layer 3
weights_conv3 = tf.get_variable("conv3_weights",
                               shape = [5,5,128,128],
                               dtype = tf.float32,
                                initializer = tf.contrib.layers.xavier_initializer())
bias_conv3 = tf.get_variable("conv3_bias",
                             shape = [128],
                            dtype = tf.float32,
                            initializer = tf.contrib.layers.xavier_initializer())

conv3 = tf.nn.conv2d(conv2_out,weights_conv3, strides = [1,2,2,1], padding = 'VALID')
conv3_out = tf.nn.relu(conv3)


In [6]:
#max pool1
pool1_out = tf.layers.max_pooling2d(conv3_out,pool_size = [4,4], strides = [1,1], padding = 'VALID')
print(pool1_out)

Tensor("max_pooling2d/MaxPool:0", shape=(?, 6, 7, 128), dtype=float32)


In [12]:
#unstacking the input
reshape_input = tf.reshape(conv3_out,[-1,int(pool1_out.shape[1])*int(pool1_out.shape[2]),int(pool1_out.shape[3])])
print(reshape_input)
input_lstm = tf.unstack(reshape_input,reshape_input.shape[2],2)
# print(input_lstm)

Tensor("Reshape_6:0", shape=(?, 42, 128), dtype=float32)


In [8]:
lstm_fw1 = tf.contrib.rnn.LSTMCell( 64, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=3456, dtype=tf.float32))
lstm_fw2 = tf.contrib.rnn.LSTMCell( 64, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=3457, dtype=tf.float32))

lstm_fw = tf.contrib.rnn.MultiRNNCell((lstm_fw1, lstm_fw2))


lstm_back1 = tf.contrib.rnn.LSTMCell( 64, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=3458, dtype=tf.float32))
lstm_back2 = tf.contrib.rnn.LSTMCell( 64, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=3459, dtype=tf.float32))

lstm_back = tf.contrib.rnn.MultiRNNCell((lstm_back1, lstm_back2))

blstm_out, state_fw, state_bw = tf.contrib.rnn.stack_bidirectional_rnn( [lstm_fw], [lstm_back], input_lstm, dtype = tf.float32)

In [9]:
#print(blstm_out[-1])
#dense layer 1

dense_1 = 512
dense_1_weights = tf.get_variable( "dense_1_weights",
                                  shape = [ 128, dense_1],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

dense_1_bias = tf.get_variable( "dense_1_bias",
                                    shape = [dense_1],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

dense_1_dnn = tf.matmul( blstm_out[-1],dense_1_weights) + dense_1_bias
dense_1_out = tf.nn.relu(dense_1_dnn)
print(dense_1_out)

Tensor("Relu_3:0", shape=(?, 512), dtype=float32)


In [10]:
#dense layer 2

dense_2 = 512
dense_2_weights = tf.get_variable( "dense_2_weights",
                                  shape = [ dense_1, dense_2],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

dense_2_bias = tf.get_variable( "dense_2_bias",
                                    shape = [dense_2],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

dense_2_dnn = tf.matmul(dense_1_out,dense_2_weights) + dense_2_bias
dense_2_out = tf.nn.relu(dense_2_dnn)
print(dense_2_out)

Tensor("Relu_4:0", shape=(?, 512), dtype=float32)


In [11]:
dense_3 = 256
dense_3_weights = tf.get_variable("dense_3_weights",
                                 shape = [dense_2,dense_3],
                                 dtype = tf.float32,
                                 initializer = tf.contrib.layers.xavier_initializer())
dense_3_bias = tf.get_variable( "dense_3_bias",
                                    shape = [dense_3],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

dense_3_dnn = tf.matmul(dense_2_out,dense_3_weights) + dense_3_bias
dense_3_out = tf.nn.relu(dense_3_dnn)
print(dense_3_out)

Tensor("Relu_5:0", shape=(?, 256), dtype=float32)


In [12]:
#output layer

final = 2
final_weights = tf.get_variable( "final_weights",
                                  shape = [ dense_3, final],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

final_bias = tf.get_variable( "final_bias",
                                    shape = [final],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

final_dnn = tf.matmul( dense_3_out, final_weights) + final_bias
final_out = tf.nn.softmax(final_dnn)
print(final_out)

Tensor("Softmax:0", shape=(?, 2), dtype=float32)


In [13]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = final_dnn))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

#correct_predictions = tf.equal( tf.argmax(final_dnn, 1), tf.argmax(output_y, 1))
#accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [14]:
#initializing tensors using sessoion
session = tf.InteractiveSession()

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)

features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\overlapData_90D'
development_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\devoverlapData_90D'
eval_features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evalcontextData_90D'
model_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Models_90D_CNNRNN_3c_1p_2lstm_3d'

dev_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Dev_sp_90D_CNNRNN_3c_1p_2lstm_3d'
eval_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Eval_sp_90D_CNNRNN_3c_1p_2lstm_3d'


In [15]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [16]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = (len(list(os.listdir(test_path))))//2
    
    for folders in range(1, indexs+1):
        features = np.load(os.path.join(test_path, "features_"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels_"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(final_out, feed_dict = {x:features})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_genuine = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_genuine.append(scores[i])
            prob_genuine.append(probabilities[i])
            
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_genuine = np.array(scores_genuine)
    scores_spoof = np.array(scores_spoof)
    
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    #print(scores_genuine.shape,scores_spoof.shape)
    
    a,b = cal_eer(scores_genuine, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
        
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_genuine, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [17]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,50):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ x:features, y:labels })
        
    save_path = saver.save(session, os.path.join(model_path,"Model"+str(i+1)))
    
    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model1 is not in all_model_checkpoint_paths. Manually adding it.
iteration   1
FRR   0.31842105263157894  FAR    0.31894736842105265    mid    0.0700342941284
 develop eer   0.31842105263157894

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model2 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
FRR   0.19210526315789472  FAR    0.19263157894736843    mid    -0.0254134852883
 develop eer   0.19210526315789472

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model3 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3
FRR   0.15921052631578947  FAR    0.16    mid    -0.300084425571
 develop eer   0.15921052631578947

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model

iteration   28
FRR   0.125  FAR    0.12526315789473685    mid    -1.71984087855
 develop eer   0.125

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model29 is not in all_model_checkpoint_paths. Manually adding it.
iteration   29
FRR   0.125  FAR    0.12526315789473685    mid    -0.0281494182535
 develop eer   0.125

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model30 is not in all_model_checkpoint_paths. Manually adding it.
iteration   30
FRR   0.12763157894736843  FAR    0.12842105263157894    mid    0.581321323033
 develop eer   0.12763157894736843

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model31 is not in all_model_checkpoint_paths. Manually adding it.
iteration   31
FRR   0.12894736842105264  FAR    0.12947368421052632    mid    0.81999386609
 develop eer   0.12894736842105264

INFO:tensorflow:C:\

KeyboardInterrupt: 

In [19]:
saver.restore(session,os.path.join(model_path,"Model"+str(34)))

for i in range(34,40):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ x:features, y:labels })
        
    save_path = saver.save(session, os.path.join(model_path,"Model"+str(i+1)))
    
    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model34
INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model35 is not in all_model_checkpoint_paths. Manually adding it.
iteration   35
FRR   0.12763157894736843  FAR    0.12842105263157894    mid    -0.491579238687
 develop eer   0.12763157894736843

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model36 is not in all_model_checkpoint_paths. Manually adding it.
iteration   36
FRR   0.12631578947368421  FAR    0.12631578947368421    mid    2.09142802387
 develop eer   0.12631578947368421

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model37 is not in all_model_checkpoint_paths. Manually adding it.
iteration   37
FRR   0.12763157894736843  FAR    0.12842105263157894    mid    2.230

In [21]:
eval_features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evaloverlapData_90D'
eval_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Eval_sp_90D_CNNRNN_3c_1p_2lstm_3d'

if not os.path.exists(eval_sp_path):
    os.makedirs(eval_sp_path)
    
with tf.Session() as session:
    for i in range(7,20):
        saver.restore(session,os.path.join(model_path,"Model"+str(i+1)))
        print("iteration  ", (i+1))
        eval_eer = get_EER(eval_features_path, (i+1), eval_sp_path)
        print("eval eer  ", eval_eer)
        print()

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model8
iteration   8
FRR   0.17950693374422189  FAR    0.17954696868754164    mid    1.2615174683
eval eer   0.17950693374422189

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model9
iteration   9
FRR   0.17950693374422189  FAR    0.17929713524317123    mid    1.36011722502
eval eer   0.17950693374422189

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model10
iteration   10
FRR   0.17796610169491525  FAR    0.17804796802131911    mid    1.3486231559
eval eer   0.17796610169491525

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model11
iteration   11
FRR   0.17796610169491525  FAR    0.17804796802131911   

In [22]:
with tf.Session() as session:
    for i in range(20,30):
        saver.restore(session,os.path.join(model_path,"Model"+str(i+1)))
        print("iteration  ", (i+1))
        eval_eer = get_EER(eval_features_path, (i+1), eval_sp_path)
        print("eval eer  ", eval_eer)
        print()

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model21
iteration   21
FRR   0.17642526964560862  FAR    0.17646568954030648    mid    -0.44990326263
eval eer   0.17642526964560862

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model22
iteration   22
FRR   0.17565485362095531  FAR    0.1757161892071952    mid    -0.640518779649
eval eer   0.17565485362095531

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model23
iteration   23
FRR   0.1810477657935285  FAR    0.1811292471685543    mid    0.4567620649
eval eer   0.1810477657935285

INFO:tensorflow:Restoring parameters from C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D_CNNRNN_3c_1p_2lstm_3d\Model24
iteration   24
FRR   0.18258859784283513  FAR    0.1826282478347768